## 3장 모형 평가와 성능 향상

### [1] 교차검증

- K-겹 교차검증

In [1]:
#1. 모듈과 함수 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.model_selection import cross_val_score

#2. 데이터 불러오기
df = pd.read_csv('Ashopping.csv', encoding = 'cp949')

#3. 변수 지정(독립변수/종속변수)
X = df[['총매출액', '1회 평균매출액', '할인권 사용 횟수']]
Y = df['평균 구매주기']

#4. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#5. 모형 생성
model = KNeighborsRegressor()

#6. 5-겹 교차검증 수행
scores = cross_val_score(model, X_train, Y_train, cv=5)
print('교차검증 점수:', scores)

교차검증 점수: [0.65777511 0.71775927 0.61704877 0.64648606 0.77413476]


- 층화 K-겹 교차검증

In [2]:
#1. 모듈과 함수 불러오기
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedKFold

#2. 변수 지정(독립변수/종속변수) 
X = df[['총매출액', '거래기간', '방문빈도']]
Y = df['이탈여부']

#3. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 모형 생성
model= KNeighborsClassifier()

#5. 층화 5-겹 교차검증 수행
skf = StratifiedKFold( n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(model, X_train, Y_train, cv=skf)
print('층화 교차검증 점수:', scores)

층화 교차검증 점수: [0.64285714 0.7        0.67857143 0.67142857 0.70714286]


### [2] 변수 선택

- 일변량 통계기반 선택

In [3]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_selection import SelectKBest, chi2

#2. 변수 지정(독립변수/종속변수)
X = df.drop(['고객ID', '이탈여부'], axis=1)
Y = df['이탈여부']

#3. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 일변량 통계기반 변수 선택
feat_selector = SelectKBest(chi2)
feat_selector.fit(X_train, Y_train)

#5. 선택된 변수 출력
feat_scores = pd.DataFrame()
feat_scores['Chi-squared stats'] = feat_selector.scores_
feat_scores['P Value'] = feat_selector.pvalues_
feat_scores['Support'] = feat_selector.get_support()
feat_scores['Attribute'] = X_train.columns
feat_scores[feat_scores['Support'] == True]

,Chi-squared stats,P Value,Support,Attribute
0,5.823528e+07,0.000000e+00,True,총매출액
1,1.779863e+01,2.455599e-05,True,구매금액대
2,5.962933e+02,1.071593e-131,True,방문빈도
3,8.186325e+06,0.000000e+00,True,1회 평균매출액
5,1.425263e+03,0.000000e+00,True,총 할인 금액
11,9.683742e+00,1.859056e-03,True,성별
13,1.459119e+04,0.000000e+00,True,거래기간
14,4.657066e+01,8.837358e-12,True,할인민감여부
16,7.904040e+00,4.932450e-03,True,Frequency
18,8.628299e+03,0.000000e+00,True,평균 구매주기


- 모형기반 선택

In [4]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier

#2. 모형 기반 변수 선택
feat_selector = SelectFromModel(RandomForestClassifier(n_estimators=100, random_state=0), threshold='median')
feat_selector.fit(X_train, Y_train)

#3. 선택된 변수 출력
feat_scores = pd.DataFrame()
feat_scores['Attribute'] = X_train.columns
feat_scores['Support'] = feat_selector.get_support()
feat_scores[feat_scores['Support'] == True]

,Attribute,Support
0,총매출액,True
2,방문빈도,True
3,1회 평균매출액,True
4,할인권 사용 횟수,True
5,총 할인 금액,True
10,거주지역,True
12,고객 나이대,True
13,거래기간,True
14,할인민감여부,True
18,평균 구매주기,True


- 반복적 변수 선택

In [5]:
#1. 모듈 및 함수 불러오기
from sklearn.feature_selection import RFE
import numpy as np

#2. 반복적 변수 선택
select = RFE(RandomForestClassifier(n_estimators=100, random_state=0))
select.fit(X_train, Y_train)

#3. 선택된 변수 출력
features_bool = np.array(select.get_support())
features = np.array(X.columns)
result = features[features_bool]
print(result)

['총매출액' '방문빈도' '1회 평균매출액' '할인권 사용 횟수' '총 할인 금액' '고객 나이대' '거래기간' '할인민감여부'
 '평균 구매주기']


### [3] 데이터 밸런싱

- 언더 샘플링

In [6]:
#1. 모듈 및 함수 불러오기
from imblearn.under_sampling import RandomUnderSampler 
from collections import Counter

#2. 랜덤 언더 샘플링 실행
X_train_under, Y_train_under = \
RandomUnderSampler(random_state=0).fit_sample(X_train, Y_train)

#3. 결과 출력
print('Original dataset shape %s' % Counter(Y))
print('sampled dataset shape %s' % Counter(Y_train))
print('Resampled dataset shape %s' % Counter(Y_train_under))

Using TensorFlow backend.


Original dataset shape Counter({0: 700, 1: 300})
sampled dataset shape Counter({0: 481, 1: 219})
Resampled dataset shape Counter({0: 219, 1: 219})


- 오버 샘플링

In [7]:
#1. 모듈 및 함수 불러오기
from imblearn.over_sampling import SMOTE

#2. SMOTE 샘플링 실행
X_train_over, Y_train_over = SMOTE(random_state=0).fit_sample(X_train, Y_train)

#3. 결과 출력
print('Original dataset shape %s' % Counter(Y))
print('sampled dataset shape %s' % Counter(Y_train))
print('Resampled dataset shape %s' % Counter(Y_train_over))

Original dataset shape Counter({0: 700, 1: 300})
sampled dataset shape Counter({0: 481, 1: 219})
Resampled dataset shape Counter({1: 481, 0: 481})


### [4] 그리드 서치

In [8]:
#1. 모듈 및 함수 불러오기
from sklearn.model_selection import GridSearchCV

#2. 변수 선택(독립변수/종속변수) 
X = df[['총매출액', '거래기간', '방문빈도']]
Y = df['이탈여부']

#3. 데이터 분할(학습용/평가용 데이터 세트)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=0)

#4. 검색 대상 인자 생성
mylist = list(range(1,50))
k_list = [x for x in mylist if x %2 !=0]
parameter_grid = {'n_neighbors':k_list}

#5. 그리드 서치 수행
grid_search = GridSearchCV(KNeighborsClassifier(), parameter_grid, cv = 10)
grid_search.fit(X_train, Y_train)

print('최적의 인자 :', grid_search.best_params_)

최적의 인자 : {'n_neighbors': 9}
